# Tick data

For optimum results this notebook should be run during the Forex trading session.

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 4002, clientId=15)

<IB connected to 127.0.0.1:4002 clientId=15>

Peer closed connection.


### Streaming tick data

Create some Forex contracts:

In [2]:
contracts = [Forex(pair) for pair in ('EURUSD', 'USDJPY', 'GBPUSD', 'USDCHF', 'USDCAD', 'AUDUSD')]
ib.qualifyContracts(*contracts)

eurusd = contracts[0]

In [3]:
eurusd

Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')

Request streaming ticks for them:

In [21]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

Wait a few seconds for the tickers to get filled.

In [28]:
ticker = ib.ticker(eurusd)


ticker

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2024, 7, 31, 21, 26, 13, 770814, tzinfo=datetime.timezone.utc), minTick=1e-05, bid=1.08269, bidSize=1000000.0, ask=1.08279, askSize=1000000.0, prevBid=1.08268, prevBidSize=2000000.0, prevAsk=1.08277, prevAskSize=2000000.0, high=1.08285, low=1.0824, close=1.0826)

The price of Forex ticks is always nan. To get a midpoint price use ``midpoint()`` or ``marketPrice()``.

The tickers are kept live updated, try this a few times to see if the price changes:

In [29]:
ticker.marketPrice()

1.08274

The following cell will start a 30 second loop that prints a live updated ticker table.
It is updated on every ticker change.

In [24]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(
    index=[c.pair() for c in contracts],
    columns=['bidSize', 'bid', 'ask', 'askSize', 'high', 'low', 'close'])

def onPendingTickers(tickers):
    for t in tickers:
        df.loc[t.contract.pair()] = (
            t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df)        

ib.pendingTickersEvent += onPendingTickers
ib.sleep(3)
ib.pendingTickersEvent -= onPendingTickers

,bidSize,bid,ask,askSize,high,low,close
EURUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDJPY,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GBPUSD,2000000.0,1.28562,1.28592,2000000.0,1.28605,1.2855,1.2857
USDCHF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDCAD,1000000.0,1.38033,1.38059,4000000.0,1.3805,1.3803,1.3808
AUDUSD,2000000.0,0.6542,0.65434,1000000.0,0.6544,0.6541,0.65415


New tick data is available in the 'ticks' attribute of the pending tickers.
The tick data will be cleared before the next update.

To stop the live tick subscriptions:

In [18]:
for contract in contracts:
    ib.cancelMktData(contract)

cancelMktData: No reqId found for contract Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')
cancelMktData: No reqId found for contract Forex('USDJPY', conId=15016059, exchange='IDEALPRO', localSymbol='USD.JPY', tradingClass='USD.JPY')
cancelMktData: No reqId found for contract Forex('GBPUSD', conId=12087797, exchange='IDEALPRO', localSymbol='GBP.USD', tradingClass='GBP.USD')
cancelMktData: No reqId found for contract Forex('USDCHF', conId=12087820, exchange='IDEALPRO', localSymbol='USD.CHF', tradingClass='USD.CHF')
cancelMktData: No reqId found for contract Forex('USDCAD', conId=15016062, exchange='IDEALPRO', localSymbol='USD.CAD', tradingClass='USD.CAD')
cancelMktData: No reqId found for contract Forex('AUDUSD', conId=14433401, exchange='IDEALPRO', localSymbol='AUD.USD', tradingClass='AUD.USD')


### Tick by Tick data ###

The ticks in the previous section are time-sampled by IB in order to cut on bandwidth. So with ``reqMktdData`` not every tick from the exchanges is sent. The promise of ``reqTickByTickData`` is to send every tick, just how it appears in the TWS Time & Sales window. This functionality is severly nerfed by a total of just three simultaneous subscriptions, where bid-ask ticks and sale ticks also use up a subscription each.

The tick-by-tick updates are available from ``ticker.tickByTicks`` and are signalled by ``ib.pendingTickersEvent`` or ``ticker.updateEvent``.

In [19]:
ticker = ib.reqTickByTickData(eurusd, 'BidAsk')
ib.sleep(2)
print(ticker)

ib.cancelTickByTickData(ticker.contract, 'BidAsk')

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2024, 7, 31, 21, 23, 28, 747581, tzinfo=datetime.timezone.utc), minTick=1e-05, bid=1.08266, bidSize=1000000.0, ask=1.08277, askSize=1000000.0, prevBid=-1.0, prevBidSize=0.0, prevAsk=-1.0, prevAskSize=0.0, high=1.08495, low=1.0802, close=1.0815, tickByTicks=[TickByTickBidAsk(time=datetime.datetime(2024, 7, 31, 21, 23, 28, 747581, tzinfo=datetime.timezone.utc), bidPrice=1.08266, askPrice=1.08277, bidSize=1000000.0, askSize=1000000.0, tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False))])


### Historical tick data

Historical tick data can be fetched with a maximum of 1000 ticks at a time. Either the start time or the end time must be given, and one of them must remain empty:

In [20]:
import datetime

start = ''
end = datetime.datetime.now()
ticks = ib.reqHistoricalTicks(eurusd, start, end, 1000, 'BID_ASK', useRth=False)

ticks[-1]

HistoricalTickBidAsk(time=datetime.datetime(2024, 7, 31, 21, 23, 15, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False), priceBid=1.08266, priceAsk=1.08277, sizeBid=1000000.0, sizeAsk=1000000.0)

In [10]:
ib.disconnect()